In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples,silhouette_score
import operator
import random
from scipy.spatial.distance import cdist
import datetime
import warnings
warnings.filterwarnings("ignore")
from connector.mosaicio import MosaicioConnector

In [3]:
class Model():
    
    def __init__(self):
        pass
    
    def get_data(self, name=None, row_count=-1, query=None):
        ioConnector = MosaicioConnector()
        dataset = ioConnector.getPandasDataFrame(name=name, row_count=row_count, query=query)
        return dataset

    def write_data(self, df=None, database="FINCORE_DB", table=None, schema="Fincore_Analytics", mode="append"):
        ioConnector = MosaicioConnector()
        writer = {'WRITER': {'user': 'Fincore_Analytics_USR',
                             'ipaddress': '172.16.33.27', 
                             "database": database,
                             'password': 'F!NCore@123',
                             'port':1433,
        #                      'wareHouse': 'MOSAIC_DECISIONS_WH', 
                             'schema': schema, 
                             'tables': table, 
                             'host': '172.16.33.27', 
                             'type': 'RDBMS', 
                             'sub_type': 'SQLSERVER',
                             'DATASET_WRITE_MODE': mode}}
        #                      'cloudPlatform':'aws'}}

        ioConnector.writePandasDataFrame(df, writer)

In [4]:
model = Model()
tbt_df = model.get_data(name = 'Transaction_Based_Relation_Score',row_count=-1,query = '''SELECT * FROM [Fincore_Analytics].[Transaction_Based_Relation_Score]''')


In [5]:
tbt_df.head()

,Entity1_Master_Id,Entity2_Master_Id,Entity1_ID,Entity2_ID,Entity1_type_Id,Entity2_type_Id,Entity1_Id_Type,Entity2_Id_Type,Entity1_Full_Name,Entity2_Full_Name,Relationship_Atrribute_Id,Relationship_Source,Gross_Amount,Num_Of_Transaction,Latest_Transaction,Last_Transaction
0,123,123,123A,123B,123,123,123,123,Saurabh,Saurabh,122,Transaction Based Relations,5168381.0,613,2022-01-06 09:02:37.1118080,2018-02-10 09:02:37.1118080
1,123,123,123A,123B,123,123,123,123,Pramod,Pramod,152,Transaction Based Relations,87868.0,127,2022-01-06 09:02:37.1118520,2011-06-29 09:02:37.1118520
2,123,123,123A,123B,123,123,123,123,Jyoti,Jyoti,142,Transaction Based Relations,7816409.0,947,2022-01-06 09:02:37.1115980,2017-01-11 09:02:37.1115980
3,123,123,123A,123B,123,123,123,123,Harish,Harish,126,Transaction Based Relations,3601260.0,186,2022-01-06 09:02:37.1116320,2004-08-17 09:02:37.1116320
4,123,123,123A,123B,123,123,123,123,Abhijeet,Abhijeet,151,Transaction Based Relations,6120425.0,835,2022-01-06 09:02:37.1119530,1998-10-13 09:02:37.1119530


In [7]:
tbt_df['Num_Days'] = pd.to_datetime(tbt_df['Latest_Transaction']) - pd.to_datetime(tbt_df['Last_Transaction'])


In [8]:
tbt_df['Num_Days'] = tbt_df['Num_Days'].apply(lambda x:x.days) 

In [9]:
tbt_df['Num_Month'] = tbt_df['Num_Days'] // 30
tbt_df['Num_Year'] = tbt_df['Num_Days'] / 365

In [10]:
tbt_df.head()

,Entity1_Master_Id,Entity2_Master_Id,Entity1_ID,Entity2_ID,Entity1_type_Id,Entity2_type_Id,Entity1_Id_Type,Entity2_Id_Type,Entity1_Full_Name,Entity2_Full_Name,Relationship_Atrribute_Id,Relationship_Source,Gross_Amount,Num_Of_Transaction,Latest_Transaction,Last_Transaction,Num_Days,Num_Month,Num_Year
0,123,123,123A,123B,123,123,123,123,Saurabh,Saurabh,122,Transaction Based Relations,5168381.0,613,2022-01-06 09:02:37.1118080,2018-02-10 09:02:37.1118080,1426,47,3.906849
1,123,123,123A,123B,123,123,123,123,Pramod,Pramod,152,Transaction Based Relations,87868.0,127,2022-01-06 09:02:37.1118520,2011-06-29 09:02:37.1118520,3844,128,10.531507
2,123,123,123A,123B,123,123,123,123,Jyoti,Jyoti,142,Transaction Based Relations,7816409.0,947,2022-01-06 09:02:37.1115980,2017-01-11 09:02:37.1115980,1821,60,4.989041
3,123,123,123A,123B,123,123,123,123,Harish,Harish,126,Transaction Based Relations,3601260.0,186,2022-01-06 09:02:37.1116320,2004-08-17 09:02:37.1116320,6351,211,17.400000
4,123,123,123A,123B,123,123,123,123,Abhijeet,Abhijeet,151,Transaction Based Relations,6120425.0,835,2022-01-06 09:02:37.1119530,1998-10-13 09:02:37.1119530,8486,282,23.249315


In [5]:
# df = pd.DataFrame()

In [96]:
# data = pd.read_csv(r"/data/Summary_Synthetic_Dataset.csv")

In [97]:
# samp_df = pd.DataFrame()

In [98]:
# samp_df['Entity1_Master_Id'] = [123 for i in range(data.shape[0])]
# samp_df['Entity2_Master_Id'] = [123 for i in range(data.shape[0])]
# samp_df['Entity1_ID'] = ['123A' for i in range(data.shape[0])]
# samp_df['Entity2_ID'] = ['123B' for i in range(data.shape[0])]
# samp_df['Entity1_type_Id'] = [123 for i in range(data.shape[0])]
# samp_df['Entity2_type_Id'] = [123 for i in range(data.shape[0])]
# samp_df['Entity1_Id_Type'] = [123 for i in range(data.shape[0])]
# samp_df['Entity2_Id_Type'] = [123 for i in range(data.shape[0])]
# samp_df['Entity1_Full_Name'] = data['Entity_Name']
# samp_df['Entity2_Full_Name'] = data['Entity_Name']
# samp_df['Relationship_Atrribute_Id'] = [random.choice([122,123,124,125,126,140,141,142,143,144,151,152,153,154,155]) for i in range(data.shape[0])]
# samp_df['Relationship_Source'] = ['Transaction Based Relations' for i in range(data.shape[0])]
# samp_df['Gross_Amount'] = data['Transaction_Amount']
# samp_df['Num_Of_Transaction'] = [random.choice([5,9,3,76,34,17,61,56,3,10,23,45,7,21,41]) + random.randint(1,1000) for i in range(data.shape[0])]
# samp_df['Latest_Transaction'] = [datetime.datetime.now() for i in range(data.shape[0])]

In [92]:
# new_d = samp_df['Latest_Transaction'][0]-datetime.timedelta(random.randint(1,10000))
# new_d

Timestamp('2014-09-16 08:58:35.085627')

In [99]:
# samp_df['Last_Transaction'] = samp_df['Latest_Transaction'].apply(lambda x:x-datetime.timedelta(random.randint(1,10000)))

In [100]:
# model.write_data(df=samp_df,database='FINCORE_DB',
#     table='Transaction_Based_Relation_Score',
#     schema='Fincore_Analytics',
#     mode='append')

In [10]:
# df['Amount'] = data['Transaction_Amount']
# df['Num_Of_Transaction'] = [random.choice([5,9,3,76,34,17,61,56,3,10,23,45,7,21,41]) + random.randint(1,1000) for i in range(data.shape[0])]

In [11]:
# df['Relationship_Attr_Id'] = [random.choice([122,123,124,125,126,140,141,142,143,144,151,152,153,154,155]) for i in range(data.shape[0])]

In [12]:
scaler = MinMaxScaler()
X = scaler.fit_transform(tbt_df[['Gross_Amount','Num_Of_Transaction']])
km = KMeans(n_clusters = 25).fit(X)
tbt_df['Labels'] = km.labels_

In [13]:
distortions_ind = {}
inertias_ind = {}
n = 50
distortions = {}
inertias = {}
for k in range(1,10):
    
    X = tbt_df[['Gross_Amount','Num_Of_Transaction']]
    if len(X) < k:
        continue
    km = KMeans(n_clusters = k).fit(X)
    distortions[k] = sum(np.min(cdist(X,km.cluster_centers_,'euclidean'),axis = 1)) / X.shape[0]
    inertias[k] = km.inertia_
#     del X,km
    
#     distortions_ind[k] = distortions
#     inertias_ind[k] = inertias

In [14]:
# plt.plot(distortions.keys(),distortions.values())

In [15]:
X_test = [[700305,122]]
X_test = scaler.transform(X_test)
X_test[0][0]

0.06471948941159353

In [16]:
km.predict(X_test)


array([0], dtype=int32)

In [11]:
new_df = tbt_df[['Entity1_Master_Id','Entity2_Master_Id','Gross_Amount','Num_Of_Transaction','Relationship_Atrribute_Id','Num_Year']]

In [12]:
#separating the dataframe based on relationship types
cbwt_df = new_df[new_df['Relationship_Atrribute_Id'].isin([125,143,154])]
neft_rtgs_df = new_df[new_df['Relationship_Atrribute_Id'].isin([122,123,140,142,151,152])]
imps_df = new_df[new_df['Relationship_Atrribute_Id'].isin([124,142,153])]
upi_df = new_df[new_df['Relationship_Atrribute_Id'].isin([126,144,155])]

In [13]:
cbwt_df['Amount_Bin'] = pd.cut(new_df[new_df['Relationship_Atrribute_Id'].isin([125,143,154])]['Gross_Amount'],3,labels=[1,2,3])
cbwt_df['Num_Of_Transaction_Bin'] = pd.cut(new_df[new_df['Relationship_Atrribute_Id'].isin([125,143,154])]['Num_Of_Transaction'],3,labels=[1,2,3])
cbwt_df['Num_Year_Bin'] = pd.cut(new_df[new_df['Relationship_Atrribute_Id'].isin([125,143,154])]['Num_Year'],3,labels=[1,2,3])

neft_rtgs_df['Amount_Bin'] = pd.cut(new_df[new_df['Relationship_Atrribute_Id'].isin([122,123,140,142,151,152])]['Gross_Amount'],3,labels=[1,2,3])
neft_rtgs_df['Num_Of_Transaction_Bin'] = pd.cut(new_df[new_df['Relationship_Atrribute_Id'].isin([122,123,140,142,151,152])]['Num_Of_Transaction'],3,labels=[1,2,3])
neft_rtgs_df['Num_Year_Bin'] = pd.cut(new_df[new_df['Relationship_Atrribute_Id'].isin([122,123,140,142,151,152])]['Num_Year'],3,labels=[1,2,3])
                                  
imps_df['Amount_Bin'] = pd.cut(new_df[new_df['Relationship_Atrribute_Id'].isin([124,142,153])]['Gross_Amount'],3,labels=[1,2,3])
imps_df['Num_Of_Transaction_Bin'] = pd.cut(new_df[new_df['Relationship_Atrribute_Id'].isin([124,142,153])]['Num_Of_Transaction'],3,labels=[1,2,3])
imps_df['Num_Year_Bin'] = pd.cut(new_df[new_df['Relationship_Atrribute_Id'].isin([124,142,153])]['Num_Year'],3,labels=[1,2,3])

upi_df['Amount_Bin'] = pd.cut(new_df[new_df['Relationship_Atrribute_Id'].isin([126,144,155])]['Gross_Amount'],3,labels=[1,2,3])
upi_df['Num_Of_Transaction_Bin'] = pd.cut(new_df[new_df['Relationship_Atrribute_Id'].isin([126,144,155])]['Num_Of_Transaction'],3,labels=[1,2,3])
upi_df['Num_Year_Bin'] = pd.cut(new_df[new_df['Relationship_Atrribute_Id'].isin([126,144,155])]['Num_Year'],3,labels=[1,2,3])

In [14]:
bin_dict = {}
val = 0
for i in range(1,4):
    for j in range(1,4):
        for k in range(1,4):
            
            val = val+0.1
            if val >= 2.5:
                val = 2.5
            bin_dict[i,j,k] = round(val,1)
            
        
        


In [15]:
bin_dict

{(1, 1, 1): 0.1,
 (1, 1, 2): 0.2,
 (1, 1, 3): 0.3,
 (1, 2, 1): 0.4,
 (1, 2, 2): 0.5,
 (1, 2, 3): 0.6,
 (1, 3, 1): 0.7,
 (1, 3, 2): 0.8,
 (1, 3, 3): 0.9,
 (2, 1, 1): 1.0,
 (2, 1, 2): 1.1,
 (2, 1, 3): 1.2,
 (2, 2, 1): 1.3,
 (2, 2, 2): 1.4,
 (2, 2, 3): 1.5,
 (2, 3, 1): 1.6,
 (2, 3, 2): 1.7,
 (2, 3, 3): 1.8,
 (3, 1, 1): 1.9,
 (3, 1, 2): 2.0,
 (3, 1, 3): 2.1,
 (3, 2, 1): 2.2,
 (3, 2, 2): 2.3,
 (3, 2, 3): 2.4,
 (3, 3, 1): 2.5,
 (3, 3, 2): 2.5,
 (3, 3, 3): 2.5}

In [16]:
cbwt_wt = 1
neft_rtgs_wt = 0.9
imps_wt = 0.8
upi_wt = 0.7

In [17]:
cbwt_df['Relationsip_Score'] = cbwt_df.apply(lambda row:cbwt_wt*bin_dict[row['Amount_Bin'],row['Num_Of_Transaction_Bin'],row['Num_Year_Bin']],axis=1)
neft_rtgs_df['Relationsip_Score'] = neft_rtgs_df.apply(lambda row:neft_rtgs_wt*bin_dict[row['Amount_Bin'],row['Num_Of_Transaction_Bin'],row['Num_Year_Bin']],axis=1)
imps_df['Relationsip_Score'] = imps_df.apply(lambda row:imps_wt*bin_dict[row['Amount_Bin'],row['Num_Of_Transaction_Bin'],row['Num_Year_Bin']],axis=1)
upi_df['Relationsip_Score'] = upi_df.apply(lambda row:upi_wt*bin_dict[row['Amount_Bin'],row['Num_Of_Transaction_Bin'],row['Num_Year_Bin']],axis=1)

In [19]:
final_df = pd.DataFrame()
final_df = pd.concat([cbwt_df,neft_rtgs_df,imps_df,upi_df],axis=0)

In [20]:
final_df

,Entity1_Master_Id,Entity2_Master_Id,Gross_Amount,Num_Of_Transaction,Relationship_Atrribute_Id,Num_Year,Amount_Bin,Num_Of_Transaction_Bin,Num_Year_Bin,Relationsip_Score
9,123,123,9268302.0,1029,125,21.375342,3,3,3,2.50
11,123,123,1276191.0,255,125,9.646575,1,1,2,0.20
12,123,123,1592781.0,49,143,7.868493,1,1,1,0.10
15,123,123,8267397.0,521,125,9.501370,3,2,2,2.30
18,123,123,9424800.0,502,154,7.400000,3,2,1,2.20
...,...,...,...,...,...,...,...,...,...,...
2975,123,123,4645903.0,118,144,20.268493,2,1,3,0.84
2988,123,123,7467549.0,643,126,25.369863,3,2,3,1.68
2989,123,123,4587542.0,508,126,14.539726,2,2,2,0.98
2990,123,123,7715003.0,441,155,2.238356,3,2,1,1.54


In [21]:
df_to_write = pd.merge(tbt_df,final_df,on=['Entity1_Master_Id','Entity2_Master_Id'],how='inner')

In [1]:
df_to_write_copy = df_to_write.iloc[:10,:]

NameError: name 'df_to_write' is not defined

In [ ]:
df_to_write_copy['Relationship_Timestamp'] = [datetime.datetime.now() for i in range(len(df_to_write_copy))]
df_to_write_copy['Resolved_Relationship_Id'] = [i for i in range(len(df_to_write_copy))]

In [27]:
df_to_write.columns

Index(['Entity1_Master_Id', 'Entity2_Master_Id', 'Entity1_ID', 'Entity2_ID',
       'Entity1_type_Id', 'Entity2_type_Id', 'Entity1_Id_Type',
       'Entity2_Id_Type', 'Entity1_Full_Name', 'Entity2_Full_Name',
       'Relationship_Atrribute_Id_x', 'Relationship_Source', 'Gross_Amount_x',
       'Num_Of_Transaction_x', 'Labels', 'Gross_Amount_y',
       'Num_Of_Transaction_y', 'Relationship_Atrribute_Id_y', 'Amount_Bin',
       'Num_Of_Transaction_Bin', 'Relationsip_Score', 'Relationship_Timestamp',
       'Resolved_Relationship_Id'],
      dtype='object')

In [28]:
df_to_write.columns.to_list()

['Entity1_Master_Id',
 'Entity2_Master_Id',
 'Entity1_ID',
 'Entity2_ID',
 'Entity1_type_Id',
 'Entity2_type_Id',
 'Entity1_Id_Type',
 'Entity2_Id_Type',
 'Entity1_Full_Name',
 'Entity2_Full_Name',
 'Relationship_Atrribute_Id_x',
 'Relationship_Source',
 'Gross_Amount_x',
 'Num_Of_Transaction_x',
 'Labels',
 'Gross_Amount_y',
 'Num_Of_Transaction_y',
 'Relationship_Atrribute_Id_y',
 'Amount_Bin',
 'Num_Of_Transaction_Bin',
 'Relationsip_Score',
 'Relationship_Timestamp',
 'Resolved_Relationship_Id']

In [29]:
df_to_write.shape

(8994001, 23)

In [30]:
temp = df_to_write.iloc[:1,[0,1,2,3,4,5,6,7,8,9,10,11,20,21,22]]

In [31]:
temp.columns

Index(['Entity1_Master_Id', 'Entity2_Master_Id', 'Entity1_ID', 'Entity2_ID',
       'Entity1_type_Id', 'Entity2_type_Id', 'Entity1_Id_Type',
       'Entity2_Id_Type', 'Entity1_Full_Name', 'Entity2_Full_Name',
       'Relationship_Atrribute_Id_x', 'Relationship_Source',
       'Relationsip_Score', 'Relationship_Timestamp',
       'Resolved_Relationship_Id'],
      dtype='object')

In [32]:
temp.rename(columns={'Relationship_Atrribute_Id_x':'Relationship_Attributes_Id','Entity1_Full_Name':'Entity1_Name','Entity2_Full_Name':'Entity2_Name'},inplace=True)

In [33]:
temp['Update_Job_Id'] = ['das']
temp['Create_Job_Id'] = ['das']

In [34]:
model.write_data(df=temp,
    database='FINCORE_DB',
    table='Resolved_Relationship',
    schema='Fincore',
    mode='append',)

NameError: name 'model' is not defined